# Negative Triangularity example pulsed equilibrium problem

In [ ]:
from IPython import get_ipython
import numpy as np
import matplotlib.pyplot as plt
from BLUEPRINT.base.lookandfeel import plot_defaults
from BLUEPRINT.geometry.loop import Loop
from BLUEPRINT.geometry.parameterisations import PictureFrame
from BLUEPRINT.equilibria.run import AbInitioEquilibriumProblem
from BLUEPRINT.equilibria.profiles import DoublePowerFunc

try:
    get_ipython().run_line_magic("matplotlib", "qt")
except AttributeError:
    pass

Set some plotting defaults

In [ ]:
plt.close("all")
plot_defaults()

Make a TF coil shape and use it as an exclusion zone object

In [ ]:
TF = PictureFrame()
TF.xo["x1"] = {"value": 3.5, "lb": 3.5, "ub": 5}  # inner leg
TF.xo["x2"] = {"value": 16.2, "lb": 14, "ub": 20}  # outer leg
TF.xo["r"] = {"value": 2, "lb": 1.999, "ub": 2.001}  # Corner radius
TF.xo["z1"] = {"value": 9, "lb": 5, "ub": 15}  # Vertical height
TF.xo["z2"] = {"value": -9, "lb": -15, "ub": -3}  # vertical
TF = Loop(**TF.draw())
clip = np.where(TF.x >= 3.6)
TF = Loop(TF.x[clip], z=TF.z[clip])

Choose a flux function parameterisation (with some initial values)
The shape function parameters will be optimised to meet the integral plasma
parameters we specify later on

In [ ]:
profile = DoublePowerFunc([2, 2])

Set up an equilibrium problem (a typical sized pulsed EU-DEMO like machine)

eps BD opt 1e-3
gamma FBI 1e-15

In [ ]:
NTT = AbInitioEquilibriumProblem(
    R_0=8,
    B_0=5.2,
    A=3.2,
    Ip=15e6,
    betap=1.2,
    li=0.8,
    kappa=1.655,
    delta=-0.3,
    r_cs=2.6,
    tk_cs=0.32,
    tfbnd=TF,
    n_PF=5,
    n_CS=6,
    eqtype="SN",
    rtype="Normal",
    profile=profile,
)

Do an initial solve

In [ ]:
eqref = NTT.solve(plot=True)

Now let's optimise:
*  positions of the PF coils
*  currents of the PF and CS coils

constraining:
*  plasma shape
*  plasma integral values (I_p, beta_p, l_i)
*  coil positions         (L)
*  coil currents          (I)
*  coil forces            (F)
*  field at coils         (B)
*  pulse length           (tau_flattop)

Define some exclusion zones

In [ ]:
UP = Loop(x=[6, 12, 12, 6, 6], z=[3, 3, 14.5, 14.5, 3])
LP = Loop(x=[10, 10, 12, 22, 22, 10], z=[-6, -6, -11, -11, -6, -6])
EQ = Loop(x=[14, 22, 22, 14, 14], z=[-1.4, -1.4, 1.4, 1.4, -1.4])

Run the optimisation

In [ ]:
NTT.optimise_positions(
    max_PF_current=1.5 * 15e6,
    PF_Fz_max=350e6,
    CS_Fz_sum=300e6,
    CS_Fz_sep=250e6,
    tau_flattop=1.1 * 3600,
    v_burn=0.04,
    psi_bd=None,  # will auto-calculate breakdown flux
    pfcoiltrack=TF,
    pf_exclusions=[LP, EQ, UP],
    CS=False,
    plot=True,
    gif=False,
)

Generate a summary plot

In [ ]:
NTT.plot_summary()

Let's have a look at the plasma over the pulse

In [ ]:
print(NTT.report())

It doesn't look very good... we probably need some more PF coils in there.

Or actually..! Fix the breakdown optimiser which I think is not doing a
great job here :(

Let's see what the optimisers have done

In [ ]:
NTT.opt_report()